In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from PIL import Image

In [ ]:
image1=Image.open('../input/plant-seedlings-classification/test/0021e90e4.png')

In [ ]:
image1_size=image1.size
image1_size

In [ ]:
import glob
train=pd.DataFrame({'path':glob.glob('../input/plant-seedlings-classification/train/*/*')})
train
train['target']=train['path'].apply(lambda x:x.split('/')[4])
pd.options.display.max_colwidth=999
train

In [ ]:
train['target'].value_counts()

In [ ]:
from sklearn.model_selection import StratifiedKFold #k-fold 쓰면 되는데 stratit 이건 정답클래스의 비율이 다를 때 사용한다.
skf=StratifiedKFold(n_splits=5,shuffle=True,random_state=0) #n_split 데이터 분할 수 
for train_index, valid_index in skf.split(train,train['target']): #train['target'] 컬럼의 카테고리를 기준으로, 비율에 맞게 
    x_train=train.iloc[train_index] #데이터에 접근해서 대괄호 진행하자, x_train이 20000개라 할 시 16000개가 들어가게된다. train에서 train_index만 들고 오겠다
    x_valid=train.iloc[valid_index]

In [ ]:
#from sklearn.model_selection import train_test_split
#x_train,x_valid=train_test_split(train,test_size=0.2)
#x_train

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
idg = ImageDataGenerator(horizontal_flip=True,vertical_flip=True)
train_generator = idg.flow_from_dataframe(x_train, x_col = "path", y_col = "target", target_size = (300,300), batch_size = 32)

In [ ]:
idg2=ImageDataGenerator()
vaild_generator=idg2.flow_from_dataframe(x_valid, x_col = "path", y_col = "target", target_size = (300,300), batch_size = 32)

In [ ]:
!pip install -U git+https://github.com/qubvel/efficientnet

In [ ]:
from efficientnet.tfkeras import EfficientNetB1
from keras import Sequential
from keras.layers import * 
from keras.optimizers import SGD
model=Sequential()
model.add(EfficientNetB1(weights='imagenet',pooling='avg',include_top=False))
model.add(Dense(12,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer=SGD(lr=0.01,nesterov=True,momentum=0.9),metrics=['acc'])

from keras.callbacks import EarlyStopping,ModelCheckpoint, ReduceLROnPlateau
es=EarlyStopping(patience=5)
mc=ModelCheckpoint('best.h5',save_best_only=True)
RL=ReduceLROnPlateau(patience=3,factor=0.2) 
model.fit(train_generator,callbacks=[es,mc,RL],epochs=100)

val_loss, val_acc= 학습에 이용하지 않는 데이터 셋에 대한 점수이기 때문에 즉,val_loss/val_acc이가 점수이다. 

In [ ]:
test=pd.DataFrame({'path':glob.glob('../input/plant-seedlings-classification/test/*')})
test

In [ ]:
idg3=ImageDataGenerator()
test_generator = idg3.flow_from_dataframe(test, x_col = "path", y_col = None, target_size = (300,300), batch_size = 64, class_mode=None, shuffle=False)

In [ ]:
result=model.predict(test_generator)

In [ ]:
result

In [ ]:
sub=pd.read_csv('../input/plant-seedlings-classification/sample_submission.csv')
sub

In [ ]:
sub['file']=test['path'].apply(lambda x:x.split('/')[4])
sub

가장 높은 확률값, argmax로 

In [ ]:
result.argmax(1) #axis=1, 가장 높은 값을 가지는 class를 출력한 상태

In [ ]:
sorted(train['target'].unique())

In [ ]:
result_list=[]
for i in result.argmax(1):
    a=sorted(train['target'].unique())[i]
    result_list.append(a)
    print(i)
result_list

In [ ]:
sub['species']=result_list

In [ ]:
sub.to_csv('sub_plant.csv',index=0)